In [8]:
import pandas as pd
import polars as pl
from datetime import datetime, timedelta

In [6]:
hoy = datetime.today()
lunes = hoy - timedelta(days=hoy.weekday())
viernes = lunes + timedelta(days=4)
fecha_min = lunes.strftime("%d/%m")
fecha_max = viernes.strftime("%d/%m")
fecha_max

'19/12'

In [14]:
clubs = (pl.read_csv("https://raw.githubusercontent.com/IvoVillanueva/acb2026/refs/heads/main/logos_calendario/clubs2026.csv")
    .with_columns(
        pl.col("equipo").str.replace("Baskonia", "Baskonia Kosner"))
    .with_columns([
        pl.when(pl. col("equipo") == "Real Madrid")
          .then(pl.lit("#00529F"))
          .otherwise(pl.col("color"))
          .alias("color2"),
        
        pl.when(pl.col("equipo") == "Baskonia Kosner")
          .then(pl.lit("#000000"))
          .otherwise(pl.col("color"))
          .alias("color"),
        
        pl.when(pl.col("equipo") == "Baskonia Kosner")
          .then(pl.lit("https://static.acb.com/img/www/clubes2025/2526KosnerBaskoniaLogoNegativo.png"))
          .otherwise(pl.col("logo"))
          .alias("logo")
    ]))

In [21]:
jugadores = (
    pl.read_csv("https://raw.githubusercontent.com/IvoVillanueva/DATOS-JUGADORES-SUPERMANAGER-2025_26/refs/heads/main/data/supermanager_juagadores_2026.csv")
    .select([ pl.col("fullName").alias("player"),
    pl.col("nameTeam").alias("equipo"),
    "price", "photo", "number", "license", "position"])
)
jugadores

player,equipo,price,photo,number,license,position
str,str,f64,str,i64,str,i64
"""Giorgi Shermadini""","""La Laguna Tenerife""",1.289e6,"""0000831003_5-6_03.jpg""",19,"""EUR""",5
"""Marcelinho Huertas""","""La Laguna Tenerife""",1.016667e6,"""0000831109_5-6_03.jpg""",9,"""EUR""",1
"""Jaime Fernández""","""La Laguna Tenerife""",733000.0,"""0000831035_5-6_03.jpg""",3,"""JFL""",3
"""Pep Busquets""","""Bàsquet Girona""",665000.0,"""0000830205_5-6_03.jpg""",4,"""JFL""",3
"""Otis Livingston""","""Bàsquet Girona""",625000.0,"""0000830179_5-6_03.jpg""",11,"""EXT""",1
…,…,…,…,…,…,…
"""Arturas Butajevas""","""Unicaja""",148750.0,"""0000826741_5-6_03.jpg""",56,"""EUR""",5
"""Eugene Omoruyi""","""Kosner Baskonia""",470000.0,"""0000865950_5-6_03.jpg""",5,"""COT""",3
"""Stefan Joksimovic""","""Kosner Baskonia""",67554.0,"""0000833725_5-6_03.jpg""",77,"""EUR""",3


In [11]:
data_euroleague = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/EUROLEAGUE-BOXSCORE/refs/heads/main/data/euroleague_boxscore_2025_26.csv"
    )
    .with_columns(
        pl.col("date").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("date"),
        
        pl.col("minutes").str.replace("DNP", "0:00").alias("minutes")
    )
    .with_columns(
        pl.when(pl.col("team_name") == "FC BARCELONA")
            .then(pl.lit("Barça"))
            .when(pl.col("team_name") == "VALENCIA BASKET")
            .then(pl.lit("Valencia Basket"))
            .when(pl.col("team_name") == "REAL MADRID")
            .then(pl.lit("Real Madrid"))
            .when(pl.col("team_name") == "KOSNER BASKONIA VITORIA-GASTEIZ")
            .then(pl.lit("Baskonia Kosner"))
            .otherwise(pl.col("team_name"))
            .alias("team_name"),

        pl.when(pl.col("player").str.contains(","))
            .then(
                pl.col("player")
                .str.split(", ")
                .list.get(1).str.to_titlecase() + 
                pl.lit(" ") + 
                pl.col("player").str.split(", ").list.get(0).str.to_titlecase()
            )
            .otherwise(pl.col("player").str.to_titlecase())
            .alias("player"),
        
        (
            pl.col("minutes")
            .str.split(":")
            .list.get(0).cast(pl.Int64) +
            pl.when(pl.col("minutes").str.split(":").list.get(1).cast(pl.Int64) >= 30)
                .then(1)
                .otherwise(0)
        ).alias("minutes")
    )
    .filter(
        pl.col("team_name").is_in(
            ["Barça", "Valencia Basket", "Real Madrid", "Baskonia Kosner"]
        )
        & (pl.col("date") >= lunes)
        & (pl.col("date") <= viernes)
    )
    .rename({
        "isLeague": "liga",
        "date": "fecha",
        "id_match": "partido",
        "player_id": "id_jug",
        "is_starter": "titular",
        "is_playing": "juega",
        "team_name": "equipo",
        "minutes": "min",
        "points": "pts",
        "field_goals_made2": "tc2_ano",
        "field_goals_attempted2": "tc2_int",
        "field_goals_made3": "tc3_ano",
        "field_goals_attempted3": "tc3_int",
        "free_throws_made": "tl_ano",
        "free_throws_attempted": "tl_int",
        "offensive_rebounds": "reb_of",
        "defensive_rebounds": "reb_def",
        "total_rebounds": "reb",
        "assistances": "ast",
        "steals": "br",
        "turnovers": "bp",
        "blocks_favour": "tap",
        "blocks_against": "tap_con",
        "fouls_commited": "fa",
        "fouls_received": "fal_rec",
        "valuation": "val",
        "plusminus": "mm"
    })
    .select([
        "liga",
        "fecha",
        "player",
        "equipo",
        "min",
        "pts",
        "reb",
        "ast",
        "br",
        "bp",
        "tap", 
        "fa",
        "mm",
        "val"
    ])
)

data_euroleague

liga,fecha,player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val
str,datetime[μs],str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""euroleague""",2025-12-16 19:30:00,"""Trey Lyles""","""Real Madrid""",25,14,7,3,2,2,1,3,2,13
"""euroleague""",2025-12-16 19:30:00,"""Alberto Abalde""","""Real Madrid""",23,7,4,2,0,0,0,4,-1,1
"""euroleague""",2025-12-16 19:30:00,"""Facundo Campazzo""","""Real Madrid""",27,25,0,2,3,3,0,1,9,28
"""euroleague""",2025-12-16 19:30:00,"""Chuma Okeke""","""Real Madrid""",12,7,7,0,0,0,1,1,-12,13
"""euroleague""",2025-12-16 19:30:00,"""Gabriele Procida""","""Real Madrid""",4,0,0,0,0,0,0,0,-5,-1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""euroleague""",2025-12-19 19:30:00,"""Matteo Spagnolo""","""Baskonia Kosner""",12,0,4,0,0,1,1,2,-6,0
"""euroleague""",2025-12-19 19:30:00,"""Trent Forrest""","""Baskonia Kosner""",36,9,3,8,0,3,0,5,-6,7
"""euroleague""",2025-12-19 19:30:00,"""Gytis Radzevicius""","""Baskonia Kosner""",27,3,3,0,1,1,0,1,13,3


In [26]:
df_partidos = (
    data_euroleague
    .group_by(["player", "equipo"])
    .agg([
        pl.col("min").mean(),
        pl.col("pts").mean(),
        pl.col("reb").mean(),
        pl.col("ast").mean(),
        pl.col("br").mean(),
        pl.col("bp").mean(),
        pl.col("tap").mean(),
        pl.col("fa").mean(),
        pl.col("mm").mean(),
        pl.col("val").mean()
    ])
    .sort("player")
    .join(clubs, on="equipo")
    .join(jugadores, on="player")
)
df_partidos

player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val,logo,color,url,abb,color2,equipo_right,price,photo,number,license,position
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,str,i64,str,i64
"""Mario Hezonja""","""Real Madrid""",18.0,8.5,3.5,2.0,0.0,1.0,0.0,1.0,-10.5,5.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F""","""Real Madrid""",803750.0,"""0000832076_5-6_03.jpg""",11,"""JFL""",3
"""Theo Maledon""","""Real Madrid""",20.5,14.5,3.0,2.0,0.5,0.5,0.0,1.5,1.5,19.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F""","""Real Madrid""",720000.0,"""0000832158_5-6_03.jpg""",12,"""EUR""",1
"""Alex Len""","""Real Madrid""",8.0,2.0,2.0,0.0,0.0,0.5,2.0,1.0,-2.0,0.5,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F""","""Real Madrid""",562063.0,"""0000855762_5-6_03.jpg""",25,"""EUR""",5
"""Usman Garuba""","""Real Madrid""",8.0,6.0,2.0,0.0,1.0,0.0,1.0,3.0,10.0,4.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F""","""Real Madrid""",585556.0,"""0000832227_5-6_03.jpg""",16,"""JFL""",5
"""Gabriel Deck""","""Real Madrid""",21.0,5.5,3.0,1.0,0.5,1.0,0.0,1.0,14.0,7.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F""","""Real Madrid""",476667.0,"""0000831980_5-6_03.jpg""",14,"""EXT""",3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Matteo Spagnolo""","""Baskonia Kosner""",16.0,4.5,2.0,0.5,0.0,1.5,0.5,2.0,0.5,3.0,"""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN""","""#C8102E""","""Kosner Baskonia""",425556.0,"""0000833621_5-6_03.jpg""",10,"""JFL""",1
"""Khalifa Diop""","""Baskonia Kosner""",24.5,9.0,6.0,2.0,1.0,0.0,1.0,3.5,21.0,16.5,"""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN""","""#C8102E""","""Kosner Baskonia""",326846.0,"""0000833540_5-6_03.jpg""",18,"""JFL""",5
"""Markus Howard""","""Baskonia Kosner""",26.0,22.5,2.5,3.5,0.5,2.5,0.0,4.5,0.5,20.0,"""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN""","""#C8102E""","""Kosner Baskonia""",310000.0,"""0000833584_5-6_03.jpg""",0,"""EXT""",3
